In [1]:
a=1
b=2
c=a+b
c

3

In [2]:
%pip install roboflow

INFO: pip is looking at multiple versions of roboflow to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of roboflow to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9683 sha256=0c589983790dfe5b1191463c848a171832772e894f0c7803c715ce0b5c35d01d
  Stored in directory: c:\users\yrp-3\appdata\local\pip\cache\wheels\8b\f1\7f\5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget

   --- ------------------------------------  1/11 [python-doten

  DEPRECATION: Building 'wget' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'wget'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [4]:
from roboflow import Roboflow
rf = Roboflow(api_key="wKugMGCPQkZaOOt2pxZE")
project = rf.workspace("roboflow-universe-projects").project("basketball-players-fy4c2")
version = project.version(25)
dataset = version.download("yolov8")
                

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.134 is required but found version=8.3.155, to fix: `pip install ultralytics==8.0.134`


Extracting Dataset Version Zip to Basketball-Players-25 in yolov8:: 100%|██████████| 2404/2404 [00:01<00:00, 1328.32it/s]


In [1]:
from ultralytics import YOLO

In [2]:
import torch
print("CUDA:", torch.cuda.is_available())
print("GPU count:", torch.cuda.device_count())
print("GPU name:", torch.cuda.get_device_name(0))
print("Total Memory:", torch.cuda.get_device_properties(0).total_memory / 1024**2, "MB")


CUDA: True
GPU count: 1
GPU name: NVIDIA GeForce RTX 4060 Laptop GPU
Total Memory: 8187.5 MB


In [8]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# Training

In [ ]:
model = YOLO("models/yolov8n.pt")  # 加载预训练模型
model.train(data='../../dataset/Basketball-Players-25/data.yaml', epochs=100, imgsz=640, batch=4)

# Validation

In [14]:
# 加载训练好的模型
model = YOLO("../../models/v8l_detection_training.pt")

# 配置验证参数
results = model.val(
    data="../../dataset/Basketball-Players-25/data.yaml",  # 数据集配置文件路径
    imgsz=640,                                           # 输入图像尺寸
    batch=8,                                             # 每批次的图像数量
    project="../../runs/detect/val"                       # 验证结果保存的路径
)

# # 打印验证结果
# print("Validation Results:")

# # 打印每个类别的mAP、Precision和Recall
# for i, class_name in enumerate(results.names):  # 获取每个类别
#     mAP = results.box.ap_class_index[i]  # 每个类别的mAP@0.5:0.95
#     precision = results.box.p[i]  # 每个类别的Precision
#     recall = results.box.r[i]  # 每个类别的Recall
    
#     print(f"类别: {class_name}")
#     print(f"  mAP@0.5:0.95: {mAP:.4f}")
#     print(f"  Precision: {precision:.4f}")
#     print(f"  Recall: {recall:.4f}")

Ultralytics 8.3.155  Python-3.10.18 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 112 layers, 43,613,547 parameters, 0 gradients, 164.9 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 1596.0261.1 MB/s, size: 185.3 KB)


val: Scanning D:\AI\yolo\dataset\Basketball-Players-25\valid\labels.cache... 32 images, 0 backgrounds, 0 corrupt: 100%|██████████| 32/32 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.30it/s]


                   all         32        477      0.891      0.866      0.895      0.652
                  Ball         18         18       0.91      0.561      0.652      0.425
                  Hoop         22         22      0.889      0.955       0.93      0.722
                Period         22         23      0.907      0.848      0.941        0.7
                Player         31        253      0.961       0.85      0.926      0.674
                   Ref         28         55      0.854      0.891      0.876      0.658
            Shot Clock         19         19      0.863          1      0.988      0.667
             Team Name         10         20      0.888        0.9       0.93      0.683
           Team Points         22         45      0.913      0.844      0.909      0.681
        Time Remaining         22         22      0.839      0.945      0.904      0.655
Speed: 0.9ms preprocess, 21.0ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to ..\..\runs

# Test

In [2]:
import supervision as sv
import numpy as np

# 创建一个示例 Detections 对象
detections = sv.Detections(
    xyxy=np.array([[100, 200, 300, 400]]),
    confidence=np.array([0.95]),
    class_id=np.array([0])
)

# 查看所有属性
print(dir(detections))

# 查看第一个检测结果的内容
if len(detections) > 0:
    print(detections[0])  # 或打印具体属性
    print(detections.xyxy)
    print(detections.confidence)
    print(detections.class_id)
    print(detections.tracker_id)  # 如果有跟踪ID

['__annotations__', '__class__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__post_init__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'area', 'box_area', 'class_id', 'confidence', 'data', 'empty', 'from_azure_analyze_image', 'from_deepsparse', 'from_detectron2', 'from_easyocr', 'from_inference', 'from_lmm', 'from_mmdetection', 'from_ncnn', 'from_paddledet', 'from_sam', 'from_tensorflow', 'from_transformers', 'from_ultralytics', 'from_yolo_nas', 'from_yolov5', 'get_anchors_coordinates', 'is_empty', 'mask', 'merge', 'metadata', 'tracker_id', 'with_nmm', 'with_nms', 'xyxy']
Detections(xyxy=array([[100, 200, 300, 400]]), mask=None